In [0]:
import csv
import json
import os
from nltk import word_tokenize
from nltk.corpus import wordnet as wn
import nltk
from textblob import TextBlob
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
# read data
input_review_file = "Amazon_Unlocked_Filtered.csv"

product_name_with_review_dict = dict()

with open(input_review_file) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_index = 0
    for row in csv_reader:
        if line_index != 0 and len(row) > 0:
            if len(row[0]) > 3:
                if row[0] not in product_name_with_review_dict.keys():
                    product_name_with_review_dict[row[0]] = [row[4]]
                else:
                    product_name_with_review_dict[row[0]].append(row[4])
        line_index += 1

In [0]:
en_stop = set(nltk.corpus.stopwords.words('english'))
custom_stop_words = ['phone', 'work', 'works', 'could', 'would', 'great', 'iphone', 'samsung']
en_stop |= set(custom_stop_words)

In [0]:
def remove_non_ascii(text):
    """

    :param text: takes input as a text
    :return: replace all the non-ascii character from the input text
    """
    return ''.join([i if ord(i) < 128 else ' ' for i in text])


# creating tokens from the given string text
def tokenize(text):
    text = remove_non_ascii(text)
    lda_tokens = word_tokenize(text.lower())
    return lda_tokens


# converting words to their lemmatized form
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma


# converting each sentence to its respective tokens
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return remove_non_ascii(' '.join(tokens))

In [0]:
def topic_modelling_and_sentiment_score(product_name, reviews):
    total_words_count = sum(len(review.split()) for review in reviews)

    print product_name
    print "\n"
    if len(reviews) > 10:
        max_df = 0.5
        min_df = 0.02
    else:
        max_df = 1.0
        min_df = 1
    vec = TfidfVectorizer(max_df=max_df, min_df=min_df).fit(reviews)
    bag_of_words = vec.transform(reviews)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    top_topics_tuple = words_freq[:20]
    
    print top_topics_tuple
    print "\n"

    review_with_sentiment_dict = dict()
    for review in reviews:
        review_with_sentiment_dict[review] = TextBlob(review).sentiment.polarity

    topic_sentiment = dict()
    topic_counts = 0
    for topic, _ in top_topics_tuple:
        for review in reviews:
            if topic in review:
                if topic not in topic_sentiment.keys():
                    topic_sentiment[topic] = review_with_sentiment_dict[review]
                else:
                    topic_sentiment[topic] += review_with_sentiment_dict[review]

            topic_counts += sum(1 for _ in re.finditer(r'\b%s\b' % re.escape(topic), review))

        print topic + " - RATIO in all words %f ", float(topic_counts) / float(total_words_count)
        print topic + " - RATIO in all REVIEWS %f ", float(topic_counts) / float(len(reviews))

        topic_sentiment[topic] = float(topic_sentiment[topic]) / float(len(reviews))
        print topic + " - Sentiment %f ", topic_sentiment[topic]

    print("\n")

In [0]:
product_name_with_reviews = dict()
for key, values in product_name_with_review_dict.iteritems():
    for line in values:
        sentence = prepare_text_for_lda(line)
        if len(sentence) > 2:
            if key not in product_name_with_reviews.keys():
                produc
                
                t_name_with_reviews[key] = [sentence]
            else:
                product_name_with_reviews[key].append(sentence)

    topic_modelling_and_sentiment_score(key, product_name_with_reviews[key])

Samsung Galaxy S6 Edge G925F 64GB Unlocked GSM LTE Octa-Core Smartphone - White Pearl (International version, No Warranty)


[(u'excellent', 6.685284931383203), (u'product', 4.544388512428871), (u'perfect', 3.803777003112327), (u'really', 3.5933253544354486), (u'everything', 3.2022509468080202), (u'excelent', 2.775897645309583), (u'thank', 2.5048820119974353), (u'buy', 2.354207093402383), (u'describe', 2.2482725422343326), (u'problem', 2.1834044033489786), (u'please', 2.158876844656855), (u'seller', 2.1276205079926966), (u'cellphone', 1.798260611524745), (u'since', 1.678565195144882), (u'first', 1.6720090412703354), (u'shipping', 1.6381738324025918), (u'style', 1.59652122304949), (u'screen', 1.5399094513673914), (u'break', 1.5191260735043217), (u'recommend', 1.4808262561785743)]


excellent - RATIO in all words %f  0.0170454545455
excellent - RATIO in all REVIEWS %f  0.203389830508
excellent - Sentiment %f  0.138418079096
product - RATIO in all words %f  0.0340909090909
product - RATIO

Samsung Galaxy S6 Edge Plus G928V 32GB Verizon + GSM 4G LTE Octa-Core Smartphone w/ 16MP Camera - Gold


[(u'unlock', 3.3724586072725717), (u'perfect', 3.176558738970063), (u'verizon', 2.473057333365468), (u'receive', 1.8912923805664001), (u'thanks', 1.7708039140488347), (u'original', 1.6080283664652044), (u'brand', 1.460744031252427), (u'lock', 1.2866181086764619), (u'want', 1.2866181086764619), (u'recognize', 1.2157591479755734), (u'return', 1.1986256111293765), (u'problem', 1.1827386819188612), (u'carrier', 1.1767696695437648), (u'condition', 1.1705930714869401), (u'model', 1.0556439501377566), (u'try', 1.0526166551864398), (u'phone', 1.0508032789716215), (u'tmobile', 1.0), (u'recieve', 1.0), (u'screen', 1.0)]


unlock - RATIO in all words %f  0.0226757369615
unlock - RATIO in all REVIEWS %f  0.238095238095
unlock - Sentiment %f  0.0422335600907
perfect - RATIO in all words %f  0.0385487528345
perfect - RATIO in all REVIEWS %f  0.404761904762
perfect - Sentiment %f  0.12941468254
ve

Apple iPhone 6 64GB Space Gray - (Verizon Wireless)


[(u'product', 13.412192941482516), (u'happy', 10.370025271124419), (u'verizon', 10.055155223355182), (u'seller', 9.609643860729815), (u'screen', 9.261159283487222), (u'apple', 8.862252444722266), (u'condition', 8.813340153247909), (u'everything', 8.575346992470003), (u'buy', 8.00599754406465), (u'price', 7.7387770907098545), (u'awesome', 7.6162754818030045), (u'problem', 7.409693475844877), (u'purchase', 7.250645231468696), (u'exactly', 6.992736625921968), (u'brand', 6.918255206741121), (u'arrive', 6.897130006527413), (u'look', 6.762580399282731), (u'receive', 6.73805485247832), (u'shipping', 6.689969120788431), (u'thank', 6.297825739411842)]


product - RATIO in all words %f  0.0176308539945
product - RATIO in all REVIEWS %f  0.131147540984
product - Sentiment %f  0.0331144125683
happy - RATIO in all words %f  0.0269972451791
happy - RATIO in all REVIEWS %f  0.200819672131
happy - Sentiment %f  0.0413869668583
verizon - RATIO in al

describe - RATIO in all words %f  0.0170648464164
describe - RATIO in all REVIEWS %f  0.10752688172
describe - Sentiment %f  0.0487694145759
problem - RATIO in all words %f  0.0358361774744
problem - RATIO in all REVIEWS %f  0.225806451613
problem - Sentiment %f  0.0399193548387
arrive - RATIO in all words %f  0.061433447099
arrive - RATIO in all REVIEWS %f  0.387096774194
arrive - Sentiment %f  0.0298860727087
awesome - RATIO in all words %f  0.0750853242321
awesome - RATIO in all REVIEWS %f  0.47311827957
awesome - Sentiment %f  0.0810035842294
unlock - RATIO in all words %f  0.0904436860068
unlock - RATIO in all REVIEWS %f  0.569892473118
unlock - Sentiment %f  0.0010752688172
service - RATIO in all words %f  0.110921501706
service - RATIO in all REVIEWS %f  0.698924731183
service - Sentiment %f  0.034178187404
happy - RATIO in all words %f  0.129692832765
happy - RATIO in all REVIEWS %f  0.817204301075
happy - Sentiment %f  0.0642771804062
galaxy - RATIO in all words %f  0.14675767

[(u'excellent', 2.845296663806902), (u'accessory', 2.375015705131292), (u'product', 2.2726747177273072), (u'everything', 2.1799356232779266), (u'thank', 2.136626368769482), (u'expect', 2.077458008683288), (u'excelent', 1.777992768463318), (u'apple', 1.7207831844958164), (u'describe', 1.2158147273908617), (u'disappoint', 1.2158147273908617), (u'recommend', 1.1100506516465518), (u'conditions', 1.1100506516465518), (u'customer', 1.0060319838466194), (u'assistance', 1.0060319838466194), (u'exelent', 1.0), (u'headphone', 0.9176629354822471), (u'expectation', 0.8682479509280583), (u'seem', 0.7897018738358044), (u'store', 0.7071067811865476), (u'ghost', 0.7071067811865476)]


excellent - RATIO in all words %f  0.0235294117647
excellent - RATIO in all REVIEWS %f  0.153846153846
excellent - Sentiment %f  0.153846153846
accessory - RATIO in all words %f  0.0470588235294
accessory - RATIO in all REVIEWS %f  0.307692307692
accessory - Sentiment %f  0.0238095238095
product - RATIO in all words %f  

...

Apple iPhone 4 32GB (Black) - Verizon


[(u'purchase', 18.464416283786015), (u'product', 17.19910504802016), (u'price', 17.003495701992705), (u'problem', 16.780271211097187), (u'happy', 15.867422172879639), (u'buy', 14.95274813547935), (u'perfect', 14.8048973703327), (u'condition', 14.037981417318473), (u'button', 13.339901060229298), (u'verizon', 12.888544486533723), (u'apple', 12.852538448401413), (u'screen', 12.711579623481018), (u'receive', 12.556028944762728), (u'describe', 12.423238890690442), (u'thank', 11.377270378244905), (u'seller', 11.339003401978413), (u'working', 11.317095877358891), (u'arrive', 11.028028101176387), (u'excellent', 10.780391065808427), (u'scratch', 10.364855792428784)]


purchase - RATIO in all words %f  0.0134474327628
purchase - RATIO in all REVIEWS %f  0.109181141439
purchase - Sentiment %f  0.0301073945256
product - RATIO in all words %f  0.02597799511
product - RATIO in all REVIEWS %f  0.210918114144
product - Sentiment %f  0.0254889912954
price - RATI

quick - RATIO in all words %f  0.2375
quick - RATIO in all REVIEWS %f  1.20634920635
quick - Sentiment %f  0.015873015873
upgrade - RATIO in all words %f  0.24375
upgrade - RATIO in all REVIEWS %f  1.2380952381
upgrade - Sentiment %f  0.0
exactly - RATIO in all words %f  0.25
exactly - RATIO in all REVIEWS %f  1.26984126984
exactly - Sentiment %f  0.00793650793651
things - RATIO in all words %f  0.25625
things - RATIO in all REVIEWS %f  1.30158730159
things - Sentiment %f  0.0


Apple 64GB iPhone 4S White (Factory Unlocked)


[(u'thank', 1.923018945039335), (u'seller', 1.395807669438785), (u'unlock', 1.294186278383556), (u'brand', 1.2248183113478002), (u'apple', 1.1915778738099545), (u'battery', 1.0901861055655748), (u'product', 1.0055610599839215), (u'phone', 0.9700075637807146), (u'really', 0.8825747084343655), (u'order', 0.858048132193793), (u'item', 0.8558054095497345), (u'everything', 0.7844598922068003), (u'using', 0.7822149528763762), (u'seal', 0.7653826889841302), (u'still', 0.

Apple iPhone 4 A1332 32GB White (GSM Unlocked)


[(u'excellent', 16.984294945069216), (u'product', 16.358847762394067), (u'apple', 13.475420035903467), (u'problem', 12.348589628799052), (u'working', 11.881688030351304), (u'buy', 10.191303102266769), (u'unlock', 9.821883739297661), (u'condition', 9.760384956181591), (u'perfect', 9.59907134044475), (u'seller', 9.014518794255547), (u'button', 8.274537075469013), (u'really', 7.968587317478246), (u'battery', 7.8164814396922395), (u'receive', 7.4647629418091075), (u'purchase', 6.56577442521487), (u'return', 6.525686978534048), (u'describe', 6.495463309369671), (u'camera', 6.488149406822475), (u'quality', 6.251980207769473), (u'look', 6.18487702699813)]


excellent - RATIO in all words %f  0.00803212851406
excellent - RATIO in all REVIEWS %f  0.0962962962963
excellent - Sentiment %f  0.0778884479718
product - RATIO in all words %f  0.0228606734631
product - RATIO in all REVIEWS %f  0.274074074074
product - Sentiment %f  0.028251579953
apple -

Apple iPhone 4s 8GB Unlocked GSM Smartphone w/ Siri, iCloud and 8MP Camera - Black


[(u'unlock', 23.411358853582467), (u'perfect', 11.100994579490301), (u'excellent', 10.801711777157827), (u'condition', 10.573625216039392), (u'excelente', 9.0), (u'lock', 8.451637178251136), (u'problem', 8.297722569926577), (u'excelent', 7.7389464688659055), (u'charge', 7.476095038938023), (u'battery', 7.2849568199784525), (u'thank', 7.138845310446696), (u'product', 7.128038769428751), (u'seller', 7.0828934190096655), (u'brand', 6.829467671359831), (u'working', 6.364468581344577), (u'mobile', 6.168249473454776), (u'love', 5.98915741953482), (u'advertise', 5.875143285532419), (u'apple', 5.6888283552256595), (u'happy', 5.40130240101389)]


unlock - RATIO in all words %f  0.0528756957328
unlock - RATIO in all REVIEWS %f  0.267605633803
unlock - Sentiment %f  0.0239732841493
perfect - RATIO in all words %f  0.0677179962894
perfect - RATIO in all REVIEWS %f  0.342723004695
perfect - Sentiment %f  0.08116197

KeyError: 'Samsung Galaxy S6 SM-G920F 32GB Unlocked GSM Smartphone - International Version, No Warranty (White Pearl)'